In [ ]:
"""
Revised Pseudocode: 
1) Find all faces of soma subtract all of the somas 
2) Split the rest of the mesh up into significant mesh pieces and
then find connectivity of which mesh piece touches which soma (can be multiple)
3) Skeletonize each one of the branches
4) --- if there are merge branches try to find the point where 
the merge error was encountered and do m - 1 cuts for 
number of somas it is touching
: if can't do good cut then just mark it as an error 
and don't include in the analysis

** could still classify it (so maybe include it still)

5) For each branch with a skeleton: within a certain range look
branching point that has large non-overlapping branch segments 
(can account for unintended loops), and split those into 
different meshes by backtracking from the skeleton to find a good cut
(may have to go out a litte on each of the limbs and then do the 
projection algorithm from the skeleton), this is so can cleanly
split the mesh pieces

Result: would have a lot of meshpieces

"""

In [3]:
import sys
sys.path.append('../meshAfterParty')
print(sys.path)

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/usr/local/lib/python3.6/dist-packages', '/src/CGAL/cgal_segmentation', '/src/CGAL/cgal_skeleton', '/src/minnie-config', '/notebooks/pipeline/python', '/notebooks/neuron_mesh_tools/meshAfterParty/cgal_skeleton_param', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '../meshAfterParty']


In [4]:
#from skeleton_utils import *
#from soma_extraction_utils import * 

import skeleton_utils as sk
import soma_extraction_utils as sm
import trimesh_utils as tu
import trimesh

from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np
from pathlib import Path

import time
import os
import pathlib

from tqdm.notebook import tqdm

# Load in the Neuron

In [5]:
current_neuron = trimesh.load_mesh("../test_neurons/multi_soma_example.off")
segment_id = 12345

# Do the Soma Detection (but isn't the original soma faces

In [7]:
soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8734
xvfb-run -n 8734 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345.off -o /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345_decimated.off -s /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/decimation_meshlab_25657756.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284334, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284334, 3))>
pre_largest_mesh_path = /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345_decimated_largest_piece.off
IN INPUT FILE VALID

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(151132, 3), faces.shape=(302272, 3))>, <trimesh.Trimesh(vertices.shape=(42551, 3), faces.shape=(85098, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(151132, 3), faces.shape=(302272, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 6149
xvfb-run -n 6149 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/decimation_meshlab_25472879.mls
done exporting decimated mesh: neuron_12345_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and 

face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/neuron_mesh_tools/Platinum_Compartment/Poisson_temp/neuron_907216.off
removed temporary output file: /notebooks/neuron_mesh_tools/Platinum_Compartment/Poisson_temp/neuron_907216_poisson.off
/notebooks/neuron_mesh_tools/Platinum_Compartment/Poisson_temp/poisson_931762.mls is being deleted....
Inside sphere validater: ratio_val = 2.882922788675577
----- working on mesh after poisson #1: <trimesh.Trimesh(vertices.shape=(42551, 3), faces.shape=(85098, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8188
xvfb-run -n 8188 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/neuron_mesh_tools/Platinum_Compartment/12345/decimation_meshlab_25472879.mls
done exporting deci

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/neuron_mesh_tools/Platinum_Compartment/Poisson_temp/neuron_546835.off
removed temporary output file: /notebooks/neuron_mesh_tools/Platinum_Compartment/Poisson_temp/neuron_546835_poisson.off
/notebooks/neuron_mesh_tools/Platinum_Compartment/Poisson_temp/poisson_133510.mls is being deleted....
Inside sphere validater: ratio_val = 3.0502651518210446



 Total time for run = 115.93357729911804


In [8]:
total_soma = tu.combine_meshes(soma_mesh_list)
total_soma.show()

In [9]:
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = sm.find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

Soma List = [<trimesh.Trimesh(vertices.shape=(3177, 3), faces.shape=(6235, 3))>, <trimesh.Trimesh(vertices.shape=(2138, 3), faces.shape=(4236, 3))>]
soma_mesh_list_centers = [array([756915.02275732, 958987.26197671, 874182.12348127]), array([864025.20827876, 994593.71529467, 859608.41014967])]


# getting the soma submeshes that are connected to each soma and identifiying those that aren't (and eliminating any mesh pieces inside the soma)

In [10]:
main_mesh_total = current_neuron
sig_th_initial_split=15

In [11]:
#splitting the current neuron into distinct pieces
split_meshes = tu.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=sig_th_initial_split,
                            print_flag=False)



#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = sm.find_soma_centroid_containing_meshes(soma_mesh_list_centers,
                                        split_meshes)

containing_mesh_indices

{0: 0, 1: 0}

In [13]:
# filtering away any of the inside floating pieces: 
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]


#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    non_soma_touching_meshes = sm.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes,
                                    significance_threshold=sig_th_initial_split)                                                      


split_meshes # the meshes of the original mesh
containing_mesh_indices #the mapping of each soma centroid to the correct split mesh
soma_containing_meshes = sm.grouping_containing_mesh_indices(containing_mesh_indices)

soma_touching_meshes = [split_meshes[k] for k in soma_containing_meshes.keys()]

    
print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")

There were 78 pieces found after size threshold
Mesh piece 0 OUTSIDE all meshes (corrected)
Mesh piece 1 inside mesh 0 :( 
Mesh piece 2 inside mesh 1 :( 
Mesh piece 3 inside mesh 1 :( 
Mesh piece 4 OUTSIDE all meshes (corrected)
Mesh piece 5 inside mesh 0 :( 
Mesh piece 6 inside mesh 0 :( 
Mesh piece 7 inside mesh 0 :( 
Mesh piece 8 OUTSIDE all meshes (corrected)
Mesh piece 9 OUTSIDE all meshes (corrected)
Mesh piece 10 inside mesh 0 :( 
Mesh piece 11 inside mesh 0 :( 
Mesh piece 12 inside mesh 0 :( 
Mesh piece 13 inside mesh 0 :( 
Mesh piece 14 OUTSIDE all meshes (corrected)
Mesh piece 15 OUTSIDE all meshes (corrected)
Mesh piece 16 inside mesh 1 :( 
Mesh piece 17 inside mesh 0 :( 
Mesh piece 18 inside mesh 1 :( 
Mesh piece 19 OUTSIDE all meshes (corrected)
Mesh piece 20 inside mesh 0 :( 
Mesh piece 21 OUTSIDE all meshes (corrected)
Mesh piece 22 OUTSIDE all meshes (corrected)
Mesh piece 23 OUTSIDE all meshes (corrected)
Mesh piece 24 inside mesh 0 :( 
Mesh piece 25 OUTSIDE all meshes

# Getting the original soma faces and the original non-soma faces of original pieces

In [257]:
from importlib import reload
sm = reload(sm)
tu = reload(tu)

"""
for each soma touching mesh get the following:
1) original soma meshes
2) significant mesh pieces touching these somas
3) The soma connectivity to each of the significant mesh pieces
-- later will just translate the 


Process: 

1) Final all soma faces (through soma extraction and then soma original faces function)
2) Subtact all soma faces from original mesh
3) Find all significant mesh pieces
4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all
   the available somas
Conclusion: Will have connectivity map


"""

soma_touching_mesh_data = dict()

for z,(mesh_idx, soma_idxes) in enumerate(soma_containing_meshes.items()):
    soma_touching_mesh_data[z] = dict()
    print(f"\n\n----Working on soma-containing mesh piece {z}----")

    #1) Final all soma faces (through soma extraction and then soma original faces function)
    current_mesh = split_meshes[mesh_idx]

    current_soma_mesh_list = [soma_mesh_list[k] for k in soma_idxes]

    mesh_pieces_without_soma = sm.subtract_soma(current_soma_mesh_list,current_mesh,
                                                significance_threshold=250)

    mesh_pieces_without_soma_stacked = tu.combine_meshes(mesh_pieces_without_soma)
    
    # find the original soma faces of mesh
    soma_faces = tu.original_mesh_faces_map(current_mesh,mesh_pieces_without_soma_stacked,matching=False)
    soma_meshes = current_mesh.submesh([soma_faces],append=True)
    
    # finding the non-soma original faces
    non_soma_faces = tu.original_mesh_faces_map(current_mesh,soma_meshes,matching=False)
    non_soma_stacked_mesh = current_mesh.submesh([non_soma_faces],append=True)
    
    # 3) Find all significant mesh pieces
    sig_non_soma_pieces = tu.split_significant_pieces(non_soma_stacked_mesh,significance_threshold=200)
   
    
    soma_touching_mesh_data[z]["branch_meshes"] = sig_non_soma_pieces
    
    #4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all the available somas
    # get all the seperate mesh faces

    #How to seperate the mesh faces
    seperate_soma_meshes,soma_face_components = tu.split(soma_meshes,only_watertight=False)
    soma_touching_mesh_data[z]["soma_meshes"] = seperate_soma_meshes
    
    
    soma_to_piece_connectivity = dict()
    for i,curr_soma in enumerate(seperate_soma_meshes):
        connected_mesh_pieces = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True
        )
        #print(f"soma {i}: connected_mesh_pieces = {connected_mesh_pieces}")
        soma_to_piece_connectivity[i] = connected_mesh_pieces
    
    
    soma_touching_mesh_data[z]["soma_to_piece_connectivity"] = soma_to_piece_connectivity

soma_touching_mesh_data



----Working on soma-containing mesh piece 0----

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(153651, 3), faces.shape=(308343, 3))>, <trimesh.Trimesh(vertices.shape=(87663, 3), faces.shape=(176175, 3))>, <trimesh.Trimesh(vertices.shape=(56972, 3), faces.shape=(114671, 3))>, <trimesh.Trimesh(vertices.shape=(54953, 3), faces.shape=(110120, 3))>, <trimesh.Trimesh(vertices.shape=(53735, 3), faces.shape=(107788, 3))>, <trimesh.Trimesh(vertices.shape=(51875, 3), faces.shape=(104099, 3))>, <trimesh.Trimesh(vertices.shape=(29324, 3), faces.shape=(58779, 3))>, <trimesh.Trimesh(vertices.shape=(29151, 3), faces.shape=(58543, 3))>, <trimesh.Trimesh(vertices.shape=(14725, 3), faces.shape=(29598, 3))>, <trimesh.Trimesh(vertices.shape=(3865, 3), faces.shape=(7718, 3))>, <trimesh.Trimesh(vertices.shape=(1159, 3), faces.shape=(2282, 3))>]
There were 11 pieces found after size threshold
Mesh piece 0 OUTSIDE all meshes (corre

{0: {'branch_meshes': [<trimesh.Trimesh(vertices.shape=(153651, 3), faces.shape=(308343, 3))>,
   <trimesh.Trimesh(vertices.shape=(87663, 3), faces.shape=(176175, 3))>,
   <trimesh.Trimesh(vertices.shape=(56972, 3), faces.shape=(114671, 3))>,
   <trimesh.Trimesh(vertices.shape=(54953, 3), faces.shape=(110120, 3))>,
   <trimesh.Trimesh(vertices.shape=(53735, 3), faces.shape=(107788, 3))>,
   <trimesh.Trimesh(vertices.shape=(51875, 3), faces.shape=(104099, 3))>,
   <trimesh.Trimesh(vertices.shape=(29324, 3), faces.shape=(58779, 3))>,
   <trimesh.Trimesh(vertices.shape=(29151, 3), faces.shape=(58543, 3))>,
   <trimesh.Trimesh(vertices.shape=(14725, 3), faces.shape=(29598, 3))>,
   <trimesh.Trimesh(vertices.shape=(3865, 3), faces.shape=(7718, 3))>,
   <trimesh.Trimesh(vertices.shape=(1159, 3), faces.shape=(2282, 3))>],
  'soma_meshes': array([<trimesh.Trimesh(vertices.shape=(20481, 3), faces.shape=(40659, 3))>,
         <trimesh.Trimesh(vertices.shape=(14158, 3), faces.shape=(28309, 3))>],

In [22]:
soma_touching_mesh_data[0]["branch_meshes"]

[<trimesh.Trimesh(vertices.shape=(153651, 3), faces.shape=(308343, 3))>,
 <trimesh.Trimesh(vertices.shape=(87663, 3), faces.shape=(176175, 3))>,
 <trimesh.Trimesh(vertices.shape=(56972, 3), faces.shape=(114671, 3))>,
 <trimesh.Trimesh(vertices.shape=(54953, 3), faces.shape=(110120, 3))>,
 <trimesh.Trimesh(vertices.shape=(53735, 3), faces.shape=(107788, 3))>,
 <trimesh.Trimesh(vertices.shape=(51875, 3), faces.shape=(104099, 3))>,
 <trimesh.Trimesh(vertices.shape=(29324, 3), faces.shape=(58779, 3))>,
 <trimesh.Trimesh(vertices.shape=(29151, 3), faces.shape=(58543, 3))>,
 <trimesh.Trimesh(vertices.shape=(14725, 3), faces.shape=(29598, 3))>,
 <trimesh.Trimesh(vertices.shape=(3865, 3), faces.shape=(7718, 3))>,
 <trimesh.Trimesh(vertices.shape=(1159, 3), faces.shape=(2282, 3))>]

# At this point have done the preprocessing so can start with skeletonization

In [51]:
sk = reload(sk)
import time
global_start_time = time.time()

for j,(soma_containing_mesh_idx,mesh_data) in enumerate(soma_touching_mesh_data.items()):
    print(f"\n-- Working on Soma Continaing Mesh {j}--")
    current_branches = mesh_data["branch_meshes"]
    
    #skeletonize each of the branches
    total_skeletons = []
    
    for z,branch in enumerate(current_branches):
        print(f"\n    -- Working on branch {z}--")
        curren_skeleton = sk.skeletonize_connected_branch(branch)
        #clean the skeleton
            # --------  Doing the cleaning ------- #
        clean_time = time.time()
        filter_end_node_length=5000,
        new_cleaned_skeleton = sk.clean_skeleton(curren_skeleton,
                                distance_func=sk.skeletal_distance,
                          min_distance_to_junction=filter_end_node_length,
                          return_skeleton=True,
                          print_flag=False)
        print(f"    Total time for skeleton and cleaning of branch {z}: {time.time() - clean_time}")
        total_skeletons.append(new_cleaned_skeleton)
    soma_touching_mesh_data[j]["branch_skeletons"] = total_skeletons
    
print(f"Total time for skeletonization = {time.time() - global_start_time}")


-- Working on Soma Continaing Mesh 0--

    -- Working on branch 0--
     Starting Screened Poisson
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 7421
xvfb-run -n 7421 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/Platinum_Compartment/temp/None.off -o /notebooks/neuron_mesh_tools/Platinum_Compartment/temp/None_poisson.off -s /notebooks/neuron_mesh_tools/Platinum_Compartment/temp/poisson_554087.mls
-----Time for Screened Poisson= 39.10453677177429


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

     Starting Calcification
Before mesh subtraction number of skeleton edges = 4143
Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = 308343


No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?



Total Mesh subtraction time = 123.1163


...

-----Time for Screened Poisson= 5.294917106628418
     Starting Calcification
only one skeleton so no stacking needed
Before mesh subtraction number of skeleton edges = 21
Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = 1736



Total Mesh subtraction time = 0.1183



len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.02981281280517578
The process was using a temp folder

Using Distance measure skeletal_distance


    Total time for skeleton and cleaning of branch 10: 0.02490687370300293
Total time for skeletonization = 673.5598750114441


In [74]:
# Save our current
print("hello")
saved_file = "soma_touching_mesh_data.npz"
np.savez(saved_file,soma_touching_mesh_data=soma_touching_mesh_data)

hello


# Load in the data and look at quality of skeletons

In [ ]:
"""
mesh #9 is the axon branch
"""

In [99]:
from importlib import reload
import numpy_utils as nu
import skeleton_utils as sk



current_mesh_data = nu.load_dict(saved_file)
current_mesh_data[0]["soma_to_piece_connectivity"]

{0: [1, 4, 6, 7, 8, 9], 1: [0, 1, 2, 3, 5, 10]}

In [208]:
np.linalg.norm(unique_rows[0]-unique_rows[2])

341.4241057686761

In [215]:
edges_with_coefficients.shape
weights.shape

(2025,)

array([[0.00000000e+00, 2.00000000e+00, 3.41424106e+02],
       [0.00000000e+00, 1.00000000e+00, 3.80573620e+02],
       [2.00000000e+00, 3.00000000e+00, 4.52102300e+02],
       ...,
       [2.00100000e+03, 2.00200000e+03, 2.88889235e+02],
       [2.00200000e+03, 2.00300000e+03, 4.30385072e+02],
       [2.00300000e+03, 2.00400000e+03, 3.25514270e+02]])

In [229]:
"""
#Planning algorithm for weights


xu = reload(xu)

staring_edges = branch_skeleton
                              
stitch_print=False
    

stitch_start = time.time()

all_skeleton_vertices = staring_edges.reshape(-1,3)
unique_rows,indices = np.unique(all_skeleton_vertices,return_inverse=True,axis=0)
edges_with_coefficients = indices.reshape(-1,2)

if stitch_print:
    print(f"Getting the unique rows and indices= {time.time()-stitch_start}")
stitch_start = time.time()

#create the graph from the edges
B = nx.Graph()
B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows)])
weights = np.linalg.norm(unique_rows[edges_with_coefficients[:,0]] - unique_rows[edges_with_coefficients[:,1]],axis=1)
edges_with_weights = np.hstack([edges_with_coefficients,weights.reshape(-1,1)])
B.add_weighted_edges_from(edges_with_weights)

if stitch_print:
    print(f"Putting edges into networkx graph= {time.time()-stitch_start}")
stitch_start = time.time()

# find the shortest distance between the two different subgraphs:
from scipy.spatial import distance_matrix

UG = B.to_undirected()

if stitch_print:
    print(f"Making undirected graph= {time.time()-stitch_start}")
stitch_start = time.time()

#UG.remove_edges_from(nx.selfloop_edges(UG))
UG = xu.remove_selfloops(UG)


"""

In [231]:
branch_index = 2
branch_mesh = current_mesh_data[0]["branch_meshes"][branch_index]
branch_skeleton = current_mesh_data[0]["branch_skeletons"][branch_index]
# will show a skeleton its mesh
sk = reload(sk)


import networkx as nx
sk_graph = sk.convert_skeleton_to_graph(branch_skeleton)
node_degrees = dict(sk_graph.degree)
three_nodes = [k for k,v in node_degrees.items() if v == 3]
greater_3_nodes = [k for k,v in node_degrees.items() if v > 3]


#get the vertices of those nodes
import networkx_utils as xu
xu = reload(xu)
node3_coordinates = xu.get_node_attributes(sk_graph,"coordinates",three_nodes)
greater3_coordinates = xu.get_node_attributes(sk_graph,"coordinates",greater_3_nodes)


import ipyvolume as ipv

sk.graph_skeleton_and_mesh(
                            #other_meshes=[current_neuron],
                          other_skeletons=[branch_skeleton],
                          other_scatter=[node3_coordinates,greater3_coordinates],
                          other_scatter_colors=[[1,0,0,0.5],[1,1,1,0.5]],
                          scatter_size=0.5
                        )